In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
plt.rc("font", family="AppleGothic")
plt.rc("axes", unicode_minus=False)

In [3]:
raw = pd.read_csv("data/서울시 지하철 호선별 역별 시간대별 승하차 인원 정보.csv", encoding="cp949")
raw.shape

(45945, 52)

In [4]:
raw.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202107,1호선,동대문,665,13,13154,2112,8755,6552,13558,...,7234,7,1394,0,0,0,1,0,0,20210803
1,202107,1호선,동묘앞,92,0,3197,1077,3575,4910,5452,...,1955,3,1039,0,0,0,0,0,0,20210803
2,202107,1호선,서울역,654,6,8652,6342,11542,33836,34033,...,5824,29,551,0,0,0,0,0,0,20210803
3,202107,1호선,시청,13,0,1803,4616,2797,19665,6210,...,988,6,84,0,0,0,0,0,0,20210803
4,202107,1호선,신설동,295,14,7940,3120,7834,10994,15927,...,4104,21,437,1,0,0,0,0,0,20210803


In [5]:
raw.tail()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
45940,201501,중앙선,운길산,0,0,177,56,642,403,1292,...,633,3,222,0,0,0,0,0,0,20151223
45941,201501,중앙선,원덕,1,0,292,4,415,46,537,...,383,3,157,0,0,0,0,0,0,20151223
45942,201501,중앙선,중랑,14,1,4929,424,8539,2084,22160,...,6501,86,1617,0,0,0,0,0,0,20151223
45943,201501,중앙선,팔당,0,0,161,50,368,149,1227,...,336,8,85,0,0,0,0,0,0,20151223
45944,201501,중앙선,회기,436,3,10929,3067,23317,10430,65043,...,35702,767,9314,0,0,0,0,0,0,20151223


In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45945 entries, 0 to 45944
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   사용월           45945 non-null  int64 
 1   호선명           45945 non-null  object
 2   지하철역          45945 non-null  object
 3   04시-05시 승차인원  45945 non-null  int64 
 4   04시-05시 하차인원  45945 non-null  int64 
 5   05시-06시 승차인원  45945 non-null  int64 
 6   05시-06시 하차인원  45945 non-null  int64 
 7   06시-07시 승차인원  45945 non-null  int64 
 8   06시-07시 하차인원  45945 non-null  int64 
 9   07시-08시 승차인원  45945 non-null  int64 
 10  07시-08시 하차인원  45945 non-null  int64 
 11  08시-09시 승차인원  45945 non-null  int64 
 12  08시-09시 하차인원  45945 non-null  int64 
 13  09시-10시 승차인원  45945 non-null  int64 
 14  09시-10시 하차인원  45945 non-null  int64 
 15  10시-11시 승차인원  45945 non-null  int64 
 16  10시-11시 하차인원  45945 non-null  int64 
 17  11시-12시 승차인원  45945 non-null  int64 
 18  11시-12시 하차인원  45945 non-null  int64 
 19  12시-

In [7]:
df = raw.melt(id_vars=["사용월", "호선명", "지하철역"], var_name="시간승하차", value_name="인원수")
df.head()

,사용월,호선명,지하철역,시간승하차,인원수
0,202107,1호선,동대문,04시-05시 승차인원,665
1,202107,1호선,동묘앞,04시-05시 승차인원,92
2,202107,1호선,서울역,04시-05시 승차인원,654
3,202107,1호선,시청,04시-05시 승차인원,13
4,202107,1호선,신설동,04시-05시 승차인원,295
